<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-&amp;-Inits" data-toc-modified-id="Imports-&amp;-Inits-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports &amp; Inits</a></span></li><li><span><a href="#Data-&amp;-Model" data-toc-modified-id="Data-&amp;-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data &amp; Model</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Testing</a></span><ul class="toc-item"><li><span><a href="#Ignite-Testing" data-toc-modified-id="Ignite-Testing-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Ignite Testing</a></span></li><li><span><a href="#NLPBook-Testing" data-toc-modified-id="NLPBook-Testing-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>NLPBook Testing</a></span></li></ul></li><li><span><a href="#Trained-Embeddings" data-toc-modified-id="Trained-Embeddings-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Trained Embeddings</a></span></li></ul></div>

# CBOW Training with Frankenstein Text

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import torch

from torch import nn
from torch import optim
from torch.utils.data import DataLoader

In [ ]:
from ignite.engine import Events, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [ ]:
from consts import consts
from cbow.dataset import CBOWDataset, DataContainer
from cbow.model import CBOWClassifier, ModelContainer
from cbow.trainer import IgniteTrainer
consts

## Data & Model

In [ ]:
df = pd.read_csv(consts.proc_dataset_csv)
dc = DataContainer(df, consts.vectorizer_file, consts.batch_size, is_load=True)

classifier = CBOWClassifier(dc.vocabulary_size, consts.embedding_size)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=consts.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1)

mc = ModelContainer(classifier, optimizer, loss_func, scheduler)

pbar = ProgressBar(persist=True)
metrics = {'accuracy': Accuracy(), 'loss': Loss(loss_func)}

## Training

In [ ]:
ig = IgniteTrainer(mc, dc, consts, pbar, metrics)
ig.run()

## Testing

### Ignite Testing

In [ ]:
state_dict = torch.load(consts.work_dir/'cbow_classifier.pth')
classifier.load_state_dict(state_dict)

In [ ]:
evaluator = create_supervised_evaluator(classifier, metrics=metrics)

@evaluator.on(Events.COMPLETED)
def log_testing_results(engine):
  metrics = engine.state.metrics
  print(f"Test loss: {metrics['loss']:0.3f}")
  print(f"Test accuracy: {metrics['accuracy']:0.3f}")

In [ ]:
evaluator.run(dc.test_dl)

### NLPBook Testing

In [ ]:
def compute_accuracy(y_pred, y_target):
  _, y_pred_indices = y_pred.max(dim=1)
  n_correct = torch.eq(y_pred_indices, y_target).sum().item()
  return n_correct / len(y_pred_indices) * 100

In [ ]:
running_loss = 0.
running_acc = 0.

classifier.eval()
for i, batch in enumerate(dc.test_dl):
  x,y = batch
  y_pred = classifier(x_in=x)
  
  loss = loss_func(y_pred, y)
  loss_t = loss.item()
  running_loss += (loss_t-running_loss)/(i+1)
  
  acc_t = compute_accuracy(y_pred, y)
  running_acc += (acc_t-running_acc)/(i+1)

In [ ]:
print(f"Test loss: {running_loss:0.3f}")
print(f"Test acc: {running_acc:0.3f}%")

## Trained Embeddings

In [ ]:
import pdb

In [ ]:
def pretty_print(results):
  for item in results:
    print(f"...{item[1]:0.2f} - {item[0]}")
    
def get_closest(target_word, idx_word_bidict, embedings, n=5):
  """
    Get the n closest words to the target word
  """
  # calculate distances to all other words
#   pdb.set_trace()
  word_embedding = embeddings[idx_word_bidict.inverse[target_word.lower()]]
  distances = []
  
  for idx, word in idx_word_bidict.items():
    if word == '<MASK>' or word == target_word:
      continue
    distances.append((word, torch.dist(word_embedding, embeddings[idx])))
    
  results = sorted(distances, key=lambda x: x[1])[1:n+2]
  return results

In [ ]:
word = 'dec'
try:
  idx_word_bidict.inverse[word]
except KeyError:
  print(f"Word {word} not in vocabulary")

In [ ]:
target_words = ['frankenstein', 'monster', 'hello', 'science', 'sickness', 'lonely', 'happy']
embeddings = classifier.embedding.weight.data
idx_word_bidict = dc.vocabulary.idx_token_bidict

for target_word in target_words:
  print(f"========={target_word}============")
  try:
    idx_word_bidict.inverse[target_word]
  except:
    print(f"Word {target_word} not in vocabulary")
    continue
  pretty_print(get_closest(target_word, idx_word_bidict, embeddings, n=5))

In [ ]:
word = input("Enter a word: ")
embeddings = classifier.embedding.weight.data
idx_word_bidict = dc.vocabulary.idx_token_bidict
pretty_print(get_closest(word, idx_word_bidict, embeddings))